<a href="https://colab.research.google.com/github/alirezabayatmk/BotDetection/blob/master/Persian_Twitter_User_Id_and_Profile_Metdata_Collector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tweepy as tw

consumer_key= 'sTY4g4zYoTuFknWBOtuPG8o2c'
consumer_secret= 'Js4hhoPmTTCmo7EfNCu0IbQTX1K5BGk3I8zwZqEpVxWHiuOm2Y'
access_token= '903348660695699458-4E7zV2CTovFUzmHCXMzfZES7fyrxPLF'
access_token_secret= 'DavX53h1a0TbP6rPKPzivV1hjLY45xqVQvksBp9UJz9f2'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [3]:
!pwd

/content


In [3]:
# screen_name = 'Khamenei_fa'
# ids = []
# for fid in tw.Cursor(api.followers_ids, screen_name=screen_name, count=5000,stringify_ids=True).items():
#     ids.append(fid)

In [2]:
# screen_name = 'briennakh'
# ids = []
# for fid in Cursor(api.followers_ids, screen_name=screen_name, count=5000).items():
#     ids.append(fid)

In [ ]:
import time 
import random

with open('famous_twitter.txt','r') as f:
  content = f.readlines()
  screen_name = [i.strip() for i in content]
  
all_users = []

# for id in random.sample(screen_name, 50):
for id in screen_name:
  all_users.append(id)
  print('crawling {} followers ...'.format(str(id)))
  print('number of followers: {}'.format(str(api.lookup_users(screen_name=str(id))[0]._json['followers_count'])))
  print('minimum number of rounds to crawl all: {}'.format(int(api.lookup_users(screen_name=str(id))[0]._json['followers_count'])//5000))
  ids = []
  for fid in tw.Cursor(api.followers_ids, screen_name=id, count=5000,stringify_ids=True).items():
    if len(ids) == 5000:
      print('sleeping ...')
      time.sleep(60)
      # break
    else:
      ids.append(fid)
      
  all_users.extend(ids)
  print('number of total users collected: {}'.format(len(all_users)))
  print('number of unique users collected: {}'.format(len(set(all_users))))
  print('*****************************')

In [9]:
import pickle

with open('followers_full.pkl', 'wb') as f:
  pickle.dump(all_users, f)

In [10]:
!cp followers_full.pkl /content/drive/MyDrive/new_bot_detection

In [8]:
import pickle

with open('/content/drive/MyDrive/new_bot_detection/followers_full.pkl', 'rb') as f:
  all_users_list = pickle.load(f)

all_users = list(set(all_users_list))

In [9]:
from tqdm import tqdm

info = []
for i in tqdm(range(0, len(all_users), 100)):
    try:
        chunk = all_users[i:i+100]
        info.extend(api.lookup_users(user_ids=chunk))
    except:
        # import traceback
        # traceback.print_exc()
        print('something went wrong, skipping ...')

100%|██████████| 2981/2981 [1:09:18<00:00,  1.39s/it]


In [10]:
import pickle

with open('info_full.pkl', 'wb') as f:
  pickle.dump(info, f)

In [11]:
!cp info_full.pkl /content/drive/MyDrive/new_bot_detection

In [12]:
import pandas as pd

data = [x._json for x in info]
data_text = []
data_lang = []
for i in data:
  if i.get('status'):
    data_text.append(i.get('status')['text'])
    data_lang.append(i.get('status')['lang'])
  else:
    data_text.append(None)
    data_lang.append(None)


df = pd.DataFrame(data)
df = df[['id_str', 'name', 'screen_name', 'description', 'url', 'location', 'verified', 'protected', 'geo_enabled', 'created_at', 'followers_count', 'friends_count', 'listed_count','favourites_count', 'statuses_count','default_profile', 'default_profile_image']]
df['last_tweet'] = data_text
df['last_tweet_lang'] = data_lang
df.to_csv('users_full.csv', index=False)

In [13]:
df

,id_str,name,screen_name,description,url,location,verified,protected,geo_enabled,created_at,followers_count,friends_count,listed_count,favourites_count,statuses_count,default_profile,default_profile_image,last_tweet,last_tweet_lang
0,927213933710147590,haji,haj_mostafa_j,Chera solh!!!!! Mage jangidan cheshe????,None,Islamic Republic of Iran,False,False,False,Sun Nov 05 16:40:03 +0000 2017,4,73,0,4,0,True,False,None,None
1,1442509683210612745,علیرضا ساکی,hesam618,,None,,False,False,False,Mon Sep 27 15:21:50 +0000 2021,0,7,0,0,6,True,False,@Amanj_emw 😆😆,und
2,1291676875295608834,Shitcoin Ventures,Gorgom82219621,nice people,https://t.co/a6Rt8pz71g,,False,False,False,Fri Aug 07 10:05:47 +0000 2020,180,474,2,93,826,True,False,RT @fairlaunch_pad: Lets Countdown together De...,en
3,1162789947692261376,حسین شفیعی,shafiei1985,ماندن در صف اصحاب عاشورایی امام عشق تنها با یق...,None,,False,False,False,Sat Aug 17 18:15:51 +0000 2019,190,387,0,17712,1039,True,False,RT @sokhanran1: کاش همین امروز در سفر غیر منتظ...,fa
4,1442217586087772162,Jani,Jani10788634,,None,,False,False,False,Sun Sep 26 20:00:53 +0000 2021,0,32,0,1,1,True,True,@c48111891,und
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297734,2878441025,esishoonam,EsmaeelFatemi,‏بیخیال همه,None,Tehran,False,False,False,Sat Nov 15 19:32:39 +0000 2014,8,43,0,171,66,False,False,@zedeey فقط بعد شام,ar
297735,2749496944,majid_khodadad,majid_khodadad,,None,,False,False,False,Thu Aug 21 20:24:50 +0000 2014,89,588,0,3618,230,True,False,RT @aamin_aajjj: يه نصحيت از اين پير فرزانه به...,fa
297736,1417785859320029184,Bohlool,Bohlool60094392,My bio!,None,,False,False,False,Wed Jul 21 09:58:00 +0000 2021,1,52,0,0,0,True,False,None,None
297737,1374363267506733057,حسن یزدان,Hassan_Yazdan_3,‏‏(‏‏‏‎‎‎‎‎@Hassan_Yazdan_2 اکانت اول)\n‏‎آسما...,None,,False,False,False,Tue Mar 23 14:12:11 +0000 2021,375,542,0,1087,775,True,False,@Matower93 زبون ریختن😕,fa


In [14]:
!cp users_full.csv /content/drive/MyDrive/new_bot_detection

In [ ]:
import shutil

shutil.copy("users_full.csv", "/content/drive/MyDrive/new_bot_detection")